In [127]:
from sklearn.datasets import load_boston
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import sklearn as sk
dataset = load_boston()

c:\Users\bitcamp\anaconda3\envs\tf274gpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California ho

In [128]:
x = dataset.data
y = dataset.target
x= pd.DataFrame(x)
print("x 최솟값 : ",np.min(x))
print("x 최댓값 : ",np.max(x))

x 최솟값 :  0       0.00632
1       0.00000
2       0.46000
3       0.00000
4       0.38500
5       3.56100
6       2.90000
7       1.12960
8       1.00000
9     187.00000
10     12.60000
11      0.32000
12      1.73000
dtype: float64
x 최댓값 :  0      88.9762
1     100.0000
2      27.7400
3       1.0000
4       0.8710
5       8.7800
6     100.0000
7      12.1265
8      24.0000
9     711.0000
10     22.0000
11    396.9000
12     37.9700
dtype: float64


c:\Users\bitcamp\anaconda3\envs\tf274gpu\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\bitcamp\anaconda3\envs\tf274gpu\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [129]:
# 스케일링 하기

print("x 최솟값 : ",np.min(x))
print("x 최댓값 : ",np.max(x))

x 최솟값 :  0       0.00632
1       0.00000
2       0.46000
3       0.00000
4       0.38500
5       3.56100
6       2.90000
7       1.12960
8       1.00000
9     187.00000
10     12.60000
11      0.32000
12      1.73000
dtype: float64
x 최댓값 :  0      88.9762
1     100.0000
2      27.7400
3       1.0000
4       0.8710
5       8.7800
6     100.0000
7      12.1265
8      24.0000
9     711.0000
10     22.0000
11    396.9000
12     37.9700
dtype: float64


In [130]:
# 13개의 칼럼 을 가지고있는 데이터를 조회합니다
print(x.shape) #(506,13)
# print(x)
print(y.shape) #(506,)
# print(y)


(506, 13)
(506,)


In [131]:
# 사이킥 런의 칼럼 조회기능
print(dataset.feature_names)
print(len(dataset.feature_names))


['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
13


In [132]:
# 사이킥런의 데이터 요약
# print(dataset.DESCR)

In [133]:
# 3. 모델구성
# 실습 train 0.7 이상
# 평가지표 R2 : 0.8 이상으로 뽑아내보기 / RMSE 사용

x_train, x_test,y_train,y_test = train_test_split(x,y,
    train_size=0.8,
    shuffle = True,
    random_state= 123
)

scaler = MinMaxScaler()
# 가중치 생성후
scaler.fit(x_train)
# 변환
x_train = scaler.transform(x_train)
# x_test 는 x_train 의 볌위를 따라가야합니다.
x_test = scaler.transform(x_test)


# CNN 모델 적용을 위하 리쉐이프
x_train = x_train.reshape(404,13,1,1)
x_test = x_test.reshape(102,13,1,1)

print(x_train.shape)
print(x_test.shape)


(404, 13, 1, 1)
(102, 13, 1, 1)


In [134]:
# 모델
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Flatten

# 모델
rate = 0.2
model = Sequential([
    # 5X5 이미지 채널 1개 
    # 필터 = output 노드수 
    # 자르는 필터는 2X2 사이즈 
    Conv2D(filters=64,kernel_size=(2,1),input_shape=(13,1,1),activation="relu"),

    Flatten(),
    Dense(16,activation="relu"),
    Dense(1,activation="linear")
])

model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_43 (Conv2D)          (None, 12, 1, 64)         192       
                                                                 
 flatten_19 (Flatten)        (None, 768)               0         
                                                                 
 dense_56 (Dense)            (None, 16)                12304     
                                                                 
 dense_57 (Dense)            (None, 1)                 17        
                                                                 
Total params: 12,513
Trainable params: 12,513
Non-trainable params: 0
_________________________________________________________________


In [135]:

hist = model.compile(loss="mse",optimizer="adam")

model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.25)



Epoch 1/100
10/10 [==============================] - 13s 28ms/step - loss: 561.6862 - val_loss: 589.0376
Epoch 2/100
10/10 [==============================] - 0s 7ms/step - loss: 526.6154 - val_loss: 543.7321
Epoch 3/100
10/10 [==============================] - 0s 7ms/step - loss: 474.4300 - val_loss: 474.8546
Epoch 4/100
10/10 [==============================] - 0s 6ms/step - loss: 397.7521 - val_loss: 381.2484
Epoch 5/100
10/10 [==============================] - 0s 6ms/step - loss: 299.8194 - val_loss: 274.5571
Epoch 6/100
10/10 [==============================] - 0s 6ms/step - loss: 204.3979 - val_loss: 178.9654
Epoch 7/100
10/10 [==============================] - 0s 6ms/step - loss: 136.3953 - val_loss: 126.7453
Epoch 8/100
10/10 [==============================] - 0s 6ms/step - loss: 112.3456 - val_loss: 115.2969
Epoch 9/100
10/10 [==============================] - 0s 6ms/step - loss: 110.0213 - val_loss: 112.3134
Epoch 10/100
10/10 [==============================] - 0s 6ms/step - los